In [1]:
!pip install opacus

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 216 kB 17.0 MB/s 
     |██████████████████████████████  | 834.1 MB 1.2 MB/s eta 0:00:46tcmalloc: large alloc 1147494400 bytes == 0x3ac38000 @  0x7f20553fb615 0x5d631c 0x51e4f1 0x51e67b 0x4f7585 0x49ca7c 0x4fdff5 0x49caa1 0x4fdff5 0x49ced5 0x4f60a9 0x55f926 0x4f60a9 0x55f926 0x4f60a9 0x55f926 0x5d7c18 0x5d9412 0x586636 0x5d813c 0x55f3fd 0x55e571 0x5d7cf1 0x49ced5 0x55e571 0x5d7cf1 0x49ec69 0x5d7c18 0x49ca7c 0x4fdff5 0x49ced5
     |████████████████████████████████| 890.2 MB 5.6 kB/s 
     |████████████████████████████████| 317.1 MB 32 kB/s 
     |████████████████████████████████| 557.1 MB 11 kB/s 
     |████████████████████████████████| 849 kB 74.2 MB/s 
     |████████████████████████████████| 21.0 MB 1.4 MB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.12.1+cu113
    Uninstalling torch-1.12.1+cu113:
      Successfully unin

In [2]:
import warnings
warnings.simplefilter("ignore")

MAX_GRAD_NORM = 1.2
EPSILON = 50.0
DELTA = 1e-5
EPOCHS = 20

LR = 1e-3

In [3]:
BATCH_SIZE = 512
MAX_PHYSICAL_BATCH_SIZE = 128

In [4]:
import torch
import torchvision
import torchvision.transforms as transforms

# These values, specific to the CIFAR10 dataset, are assumed to be known.
# If necessary, they can be computed with modest privacy budgets.
CIFAR10_MEAN = (0.4914, 0.4822, 0.4465)
CIFAR10_STD_DEV = (0.2023, 0.1994, 0.2010)

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(CIFAR10_MEAN, CIFAR10_STD_DEV),
])

In [5]:
from torchvision.datasets import CIFAR10

DATA_ROOT = '../cifar10'

train_dataset = CIFAR10(
    root=DATA_ROOT, train=True, download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
)

test_dataset = CIFAR10(
    root=DATA_ROOT, train=False, download=True, transform=transform)

test_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
)

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ../cifar10/cifar-10-python.tar.gz to ../cifar10
Files already downloaded and verified


In [6]:
from torchvision import models

model = models.resnet18(num_classes=10)

In [7]:
from opacus.validators import ModuleValidator

errors = ModuleValidator.validate(model, strict=False)
errors[-5:]

[opacus.validators.errors.ShouldReplaceModuleError("BatchNorm cannot support training with differential privacy. The reason for it is that BatchNorm makes each sample's normalized value depend on its peers in a batch, ie the same sample x will get normalized to a different value depending on who else is on its batch. Privacy-wise, this means that we would have to put a privacy mechanism there too. While it can in principle be done, there are now multiple normalization layers that do not have this issue: LayerNorm, InstanceNorm and their generalization GroupNorm are all privacy-safe since they don't have this property.We offer utilities to automatically replace BatchNorms to GroupNorms and we will release pretrained models to help transition, such as GN-ResNet ie a ResNet using GroupNorm, pretrained on ImageNet"),
 opacus.validators.errors.ShouldReplaceModuleError("BatchNorm cannot support training with differential privacy. The reason for it is that BatchNorm makes each sample's normal

In [8]:
model = ModuleValidator.fix(model)
ModuleValidator.validate(model, strict=False)


[]

In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = model.to(device)

In [10]:
import torch.nn as nn
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
# optimizer = optim.RMSprop(model.parameters(), lr=LR)
optimizer = optim.SGD(model.parameters(), lr=LR, momentum = 0)

In [11]:
def accuracy(preds, labels):
    return (preds == labels).mean()

In [12]:
from opacus import PrivacyEngine

privacy_engine = PrivacyEngine()


model, optimizer, train_loader = privacy_engine.make_private_with_epsilon(
    module=model,
    optimizer=optimizer,
    data_loader=train_loader,
    epochs=EPOCHS,
    target_epsilon=EPSILON,
    target_delta=DELTA,
    max_grad_norm=MAX_GRAD_NORM,
)

print(f"Using sigma={optimizer.noise_multiplier} and C={MAX_GRAD_NORM}")

Using sigma=0.37197113037109375 and C=1.2


In [13]:
import numpy as np
from opacus.utils.batch_memory_manager import BatchMemoryManager


def train(model, train_loader, optimizer, epoch, device):
    model.train()
    criterion = nn.CrossEntropyLoss()

    losses = []
    top1_acc = []
    
    with BatchMemoryManager(
        data_loader=train_loader, 
        max_physical_batch_size=MAX_PHYSICAL_BATCH_SIZE, 
        optimizer=optimizer
    ) as memory_safe_data_loader:

        for i, (images, target) in enumerate(memory_safe_data_loader):   
            optimizer.zero_grad()
            images = images.to(device)
            target = target.to(device)

            # compute output
            output = model(images)
            loss = criterion(output, target)

            preds = np.argmax(output.detach().cpu().numpy(), axis=1)
            labels = target.detach().cpu().numpy()

            # measure accuracy and record loss
            acc = accuracy(preds, labels)

            losses.append(loss.item())
            top1_acc.append(acc)

            loss.backward()
            optimizer.step()

            if (i+1) % 200 == 0:
                epsilon = privacy_engine.get_epsilon(DELTA)
                print(
                    f"\tTrain Epoch: {epoch} \t"
                    f"Loss: {np.mean(losses):.6f} "
                    f"Acc@1: {np.mean(top1_acc) * 100:.6f} "
                    f"(ε = {epsilon:.2f}, δ = {DELTA})"
                )

In [14]:
def test(model, test_loader, device):
    model.eval()
    criterion = nn.CrossEntropyLoss()
    losses = []
    top1_acc = []

    with torch.no_grad():
        for images, target in test_loader:
            images = images.to(device)
            target = target.to(device)

            output = model(images)
            loss = criterion(output, target)
            preds = np.argmax(output.detach().cpu().numpy(), axis=1)
            labels = target.detach().cpu().numpy()
            acc = accuracy(preds, labels)

            losses.append(loss.item())
            top1_acc.append(acc)

    top1_avg = np.mean(top1_acc)
    epsilon = privacy_engine.get_epsilon(DELTA)

    print(
        f"\tTest set:"
        f"Loss: {np.mean(losses):.6f} "
        f"Acc: {top1_avg * 100:.6f} "
    )
    return np.mean(top1_acc)

In [15]:
from tqdm.notebook import tqdm

for epoch in tqdm(range(EPOCHS), desc="Epoch", unit="epoch"):
    train(model, train_loader, optimizer, epoch + 1, device)

Epoch:   0%|          | 0/20 [00:00<?, ?epoch/s]

	Train Epoch: 1 	Loss: 2.417427 Acc@1: 9.411980 (ε = 12.21, δ = 1e-05)
	Train Epoch: 1 	Loss: 2.404691 Acc@1: 9.549279 (ε = 14.55, δ = 1e-05)
	Train Epoch: 2 	Loss: 2.373940 Acc@1: 9.984691 (ε = 16.79, δ = 1e-05)
	Train Epoch: 2 	Loss: 2.364996 Acc@1: 10.080225 (ε = 18.34, δ = 1e-05)
	Train Epoch: 3 	Loss: 2.347512 Acc@1: 10.190078 (ε = 19.92, δ = 1e-05)
	Train Epoch: 3 	Loss: 2.342620 Acc@1: 10.349001 (ε = 21.21, δ = 1e-05)
	Train Epoch: 4 	Loss: 2.332261 Acc@1: 10.611486 (ε = 22.61, δ = 1e-05)
	Train Epoch: 4 	Loss: 2.329345 Acc@1: 10.702702 (ε = 23.71, δ = 1e-05)
	Train Epoch: 5 	Loss: 2.318389 Acc@1: 11.219744 (ε = 24.93, δ = 1e-05)
	Train Epoch: 5 	Loss: 2.317397 Acc@1: 11.148155 (ε = 25.93, δ = 1e-05)
	Train Epoch: 6 	Loss: 2.311499 Acc@1: 11.425023 (ε = 27.06, δ = 1e-05)
	Train Epoch: 6 	Loss: 2.310363 Acc@1: 11.595002 (ε = 27.99, δ = 1e-05)
	Train Epoch: 7 	Loss: 2.305589 Acc@1: 12.242820 (ε = 29.07, δ = 1e-05)
	Train Epoch: 7 	Loss: 2.304913 Acc@1: 12.188425 (ε = 29.93, δ = 1e

In [17]:
top1_acc = test(model, test_loader, device)

	Test set:Loss: 2.255091 Acc: 16.420037 


In [18]:
print(privacy_engine.get_epsilon(DELTA))

49.99133617318924
